In [1]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingRegressor, BaggingClassifier, RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from xgboost import XGBRegressor, XGBClassifier
from sklearn import preprocessing
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error, f1_score

In [2]:
# Генерируем уникальный seed
my_code = "RomanenkoV"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [3]:
# Читаем данные из файла
example_data = pd.read_csv("datasets/Fish.csv")

In [4]:
example_data.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [5]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

32


In [6]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

95 32 32


In [7]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
num_columns = ['Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), num_columns)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['Weight', 'Length1', 'Length2', 'Length3',
                                  'Height', 'Width'])])

In [8]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [9]:
# Устанавливаем названия столбцов
column_names = num_columns + ['Species']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [10]:
# Явно укажем типы данных, это важно для xgboost
types = {
    'Weight' : 'float64',
    'Length1' : 'float64',
    'Length2' : 'float64',
    'Length3' : 'float64',
    'Height' : 'float64',
    'Width' : 'float64',
    'Species' : 'category'
}
sc_train = sc_train.astype(types)
sc_test = sc_test.astype(types)
sc_val = sc_val.astype(types)

In [11]:
# Задание №1 - анализ различных типов ансамблей решений в задаче регрессии
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html
# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn

In [37]:
n = 4
labels = random.sample(num_columns, n)

y_labels = labels[0]
x_labels = labels[1:]

print(x_labels)
print(y_labels)

['Length2', 'Weight', 'Width']
Height


In [38]:
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_labels]
y_test = sc_test[y_labels]
y_val = sc_val[y_labels]

In [39]:
# Создайте 4 различных модели с использованием следующих классов:
# BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor, XGBRegressor.
# Решите получившуюся задачу регрессии с помощью созданных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

# mean_squared_error -> min 

In [40]:
r_models=[
BaggingRegressor(), 
    RandomForestRegressor(),
    GradientBoostingRegressor(), 
    XGBRegressor()]

In [41]:
r_models=[]

#Регрессор Бэгинга
r_models.append(BaggingRegressor())

#случайный лес
r_models.append(RandomForestRegressor())

#градиентный бустинг
r_models.append(GradientBoostingRegressor())

#eXtrem Gradient Boosting
r_models.append(XGBRegressor())

In [42]:
r_models

[BaggingRegressor(),
 RandomForestRegressor(),
 GradientBoostingRegressor(),
 XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None)]

In [43]:
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [ ]:
for model in r_models:
    print(model)
    model.fit(x_train, y_train)

In [ ]:
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(model, '\t', mse)

In [ ]:
# Выбираем лучшую модель
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

In [ ]:
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

In [45]:
# Задание №2 - анализ различных типов ансамблей в задаче классификации
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn

In [46]:
# Выбираем 2 числовых переменных, которые будут параметрами элементов набора данных
# Метка класса всегда 'Species'
n = 2
x_labels = random.sample(num_columns, n)
y_label = 'Species'

print(x_labels)
print(y_label)

['Length3', 'Width']
Species


In [47]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [48]:
y_train

0     Bream
1     Bream
2     Perch
3     Perch
4     Roach
      ...  
90    Bream
91    Bream
92    Roach
93     Pike
94    Perch
Name: Species, Length: 95, dtype: category
Categories (7, object): ['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish']

In [49]:
# Создайте 4 различных модели с использованием следующих классов:
# BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, XGBClassifier
# Решите получившуюся задачу классификации с помощью созданных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

# f1_score -> max

In [50]:
le = preprocessing.LabelEncoder()
y_test

0         Smelt
1         Smelt
2         Perch
3         Roach
4         Bream
5         Perch
6         Bream
7     Whitefish
8         Perch
9         Bream
10        Perch
11        Bream
12        Roach
13         Pike
14        Bream
15       Parkki
16        Smelt
17    Whitefish
18        Perch
19         Pike
20        Perch
21         Pike
22        Bream
23       Parkki
24        Bream
25        Bream
26        Bream
27        Bream
28        Perch
29        Perch
30        Perch
31       Parkki
Name: Species, dtype: category
Categories (7, object): ['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish']

In [51]:
le.fit(y_train)
le.fit(y_val)
le.fit(y_test)

LabelEncoder()

In [52]:
y_train=le.transform(y_train)
y_val=le.transform(y_val)
y_test=le.transform(y_test)

In [53]:
y_train

array([0, 0, 2, 2, 4, 0, 0, 2, 3, 2, 3, 2, 2, 2, 6, 3, 1, 1, 2, 5, 2, 4,
       0, 2, 4, 0, 3, 2, 2, 2, 5, 4, 0, 2, 5, 4, 4, 5, 1, 1, 0, 2, 0, 2,
       3, 3, 2, 3, 2, 2, 5, 4, 2, 2, 2, 0, 5, 2, 2, 0, 2, 2, 2, 0, 2, 0,
       2, 5, 2, 0, 2, 1, 0, 6, 1, 3, 0, 5, 4, 4, 0, 4, 3, 1, 3, 2, 4, 2,
       2, 2, 0, 0, 4, 3, 2])

In [54]:
a_models=[
BaggingClassifier(), 
    RandomForestClassifier(),
    GradientBoostingClassifier(), 
    XGBClassifier()]

In [ ]:
a_models=[]

#Регрессор Бэгинга
a_models.append(BaggingClassifier())

#случайный лес
a_models.append(RandomForestClassifier())

#градиентный бустинг
a_models.append(GradientBoostingClassifier())

#eXtrem Gradient Boosting
a_models.append(XGBClassifier(use_label_encoder=False,
                              eval_metric='mlogloss'))

In [ ]:
# Обучаем модели
for model in a_models:
    print(model)
    model.fit(x_train, y_train)

In [ ]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in a_models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(model, '\t', f1)

In [ ]:
# Выбираем лучшую модель
i_max = f1s.index(max(f1s))
best_a_model = a_models[i_max]
best_a_model

In [ ]:
y_test

In [ ]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_a_model.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

In [ ]:
# я не могу обучить модель. уже 40 минут сижу :(